In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from ddml import *

In [ ]:
import glob
from grr.utils.ihdp_data import *


data_base_dir = "./data/IHDP/sim_data_redraw_T"
simulation_files = sorted(glob.glob("{}/*.csv".format(data_base_dir)))
  
# Simulation parameters
exp_name = "IHDP"
np.random.seed(0)

data_base_dir = "./data/IHDP/sim_data_redraw_T"
simulation_files = sorted(glob.glob("{}/*.csv".format(data_base_dir)))

# Generate covariates
np.random.seed(0)

true_ATEs_list = []

nn_ls_list = []
nn_ls_logit_list = []
nn_kl_list = []
nn_tl_list = []
nn_mle_list = []

rkhs_ls_list = []
rkhs_kl_list = []
rkhs_tl_list = []
rkhs_mle_list = []

result_list = []

for tr in range(len(simulation_files)):
    print("tr", tr)
    simulation_file = simulation_files[tr]
    x = load_and_format_covariates(simulation_file, delimiter=' ')
    t, y, y_cf, mu_0, mu_1 = load_other_stuff(simulation_file, delimiter=' ')
    true_ATE = np.mean(mu_1 - mu_0)
    true_ATEs_list.append(true_ATE)
    
    covariates = x
    treatment = t.T[0]
    outcome = y.T[0]
    
    # Below, we conduct each method to estimate the ATE

    ############################

    cross_fitting_folds=2

    method = "NN_RieszLearner"

    riesz_hidden_dim= 200
    reg_hidden_dim= 100
    riesz_max_iter = 200
    reg_max_iter = 100
    
    riesz_loss = "LS"
    riesz_with_D = True
    riesz_link_name = "Linear"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_ls_list.append(result)
    

    riesz_loss = "KL"
    riesz_with_D = False
    riesz_link_name = "Logit"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_kl_list.append(result)

    riesz_loss = "TL"
    riesz_with_D = False
    riesz_link_name = "Logit"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_tl_list.append(result)
    
    riesz_loss = "MLE"
    riesz_with_D = False
    riesz_link_name = "Logit"

    dd_ate = DDML_ATE()
    result = dd_ate.estimate(covariates, treatment, 
                    outcome, method=method, 
                    cross_fitting_folds = cross_fitting_folds,
                    riesz_loss=riesz_loss, 
                    riesz_with_D=riesz_with_D, 
                    riesz_link_name=riesz_link_name, 
                    riesz_hidden_dim=riesz_hidden_dim, 
                    reg_hidden_dim=reg_hidden_dim,
                    riesz_max_iter=riesz_max_iter, 
                    reg_max_iter=reg_max_iter)

    nn_mle_list.append(result)

    ##############################

    method = "RKHS_RieszLearner"

    folds = 3
    num_basis = 100


    riesz_loss = "LS"
    riesz_with_D = True
    riesz_link_name = "Linear"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_ls_list.append(result)


    riesz_loss = "KL"
    riesz_with_D = False
    riesz_link_name = "Logit"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_kl_list.append(result)


    riesz_loss = "TL"
    riesz_with_D = False
    riesz_link_name = "Logit"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_tl_list.append(result)

    riesz_loss = "MLE"
    riesz_with_D = False
    riesz_link_name = "Logit"
    is_separate = False

    result = dd_ate.estimate(covariates, treatment, outcome, method=method, cross_fitting_folds = cross_fitting_folds, riesz_loss=riesz_loss, riesz_with_D=riesz_with_D, riesz_link_name=riesz_link_name, is_separate=is_separate, folds=folds, num_basis=num_basis)
    rkhs_mle_list.append(result)
    
    np.savetxt("results3/true_ATEs_%s.csv"%exp_name, true_ATEs_list)
    np.savetxt("results3/nn_ls_%s.csv"%exp_name, nn_ls_list)
    np.savetxt("results3/nn_kl_%s.csv"%exp_name, nn_kl_list)
    np.savetxt("results3/nn_tl_%s.csv"%exp_name, nn_tl_list)
    np.savetxt("results3/nn_mle_%s.csv"%exp_name, nn_mle_list)
    np.savetxt("results3/rkhs_ls_%s.csv"%exp_name, rkhs_ls_list)
    np.savetxt("results3/rkhs_kl_%s.csv"%exp_name, rkhs_kl_list)
    np.savetxt("results3/rkhs_tl_%s.csv"%exp_name, rkhs_tl_list)
    np.savetxt("results3/rkhs_mle_%s.csv"%exp_name, rkhs_mle_list)


In [ ]:
import pandas as pd

In [ ]:
nn_kl_list

In [ ]:
nn_ls_list

In [ ]:
true_ATEs_list2 = np.loadtxt("results2/true_ATEs_IHDP.csv")

In [ ]:
true_ATEs_list = np.array(true_ATEs_list)

nn_ls_list = np.array(nn_ls_list)
nn_kl_list = np.array(nn_kl_list)
nn_tl_list = np.array(nn_tl_list)
nn_mle_list = np.array(nn_mle_list)

rkhs_ls_list = np.array(rkhs_ls_list)
rkhs_kl_list = np.array(rkhs_kl_list)
rkhs_tl_list = np.array(rkhs_tl_list)
rkhs_mle_list = np.array(rkhs_mle_list)

In [ ]:
np.max(nn_ls_list)

In [ ]:
results = np.array([
    np.nanmean(
        np.where(
            (nn_ls_list[:, :3]) ** 2 <= 60,
            (nn_ls_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (nn_kl_list[:, :3]) ** 2 <= 60,
            (nn_kl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (nn_tl_list[:, :3]) ** 2 <= 60,
            (nn_tl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (nn_mle_list[:, :3]) ** 2 <= 60,
            (nn_mle_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_ls_list[:, :3]) ** 2 <= 60,
            (rkhs_ls_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_kl_list[:, :3]) ** 2 <= 60,
            (rkhs_kl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_tl_list[:, :3]) ** 2 <= 60,
            (rkhs_tl_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    ),
    np.nanmean(
        np.where(
            (rkhs_mle_list[:, :3]) ** 2 <= 60,
            (rkhs_mle_list[:, :3] - np.array([true_ATEs_list] * 3).T) ** 2,
            np.nan
        ),
        axis=0
    )
])






In [ ]:
pd.DataFrame(results).round(3)

In [ ]:
nn_ls_conf_list = [np.mean((nn_ls_list[:, 3] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 4])), np.mean((nn_ls_list[:, 5] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 6])), np.mean((nn_ls_list[:, 7] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 8]))]
nn_ls_conf_list = [np.mean((nn_ls_list[:, 3] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 4])), np.mean((nn_ls_list[:, 5] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 6])), np.mean((nn_ls_list[:, 7] <= true_ATEs_list2) * (true_ATEs_list2 <= nn_ls_list[:, 8]))]


In [ ]:
nn_ls_list[:, 3]

In [ ]:
true_ATEs_list2

In [ ]:
nn_ls_list[:, 4]

In [ ]:
true_ATEs_list

In [ ]:
nn_ls_list[:, 3] <= true_ATEs_list

In [ ]:
nn_ls_conf_list

In [ ]:
nn_ls_conf_list = [np.mean((nn_ls_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_ls_list[:, 4])), np.mean((nn_ls_list[:, 5] <= true_ATEs_list) * (5 <= nn_ls_list[:, 6])), np.mean((nn_ls_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_ls_list[:, 8]))]
nn_kl_conf_list = [np.mean((nn_kl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_kl_list[:, 4])), np.mean((nn_kl_list[:, 5] <= true_ATEs_list) * (5 <= nn_kl_list[:, 6])), np.mean((nn_kl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_kl_list[:, 8]))]
nn_tl_conf_list = [np.mean((nn_tl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_tl_list[:, 4])), np.mean((nn_tl_list[:, 5] <= true_ATEs_list) * (5 <= nn_tl_list[:, 6])), np.mean((nn_tl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_tl_list[:, 8]))]
nn_mle_conf_list = [np.mean((nn_mle_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= nn_mle_list[:, 4])), np.mean((nn_mle_list[:, 5] <= true_ATEs_list) * (5 <= nn_mle_list[:, 6])), np.mean((nn_mle_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= nn_mle_list[:, 8]))]
rkhs_ls_conf_list = [np.mean((rkhs_ls_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_ls_list[:, 4])), np.mean((rkhs_ls_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_ls_list[:, 6])), np.mean((rkhs_ls_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_ls_list[:, 8]))]
rkhs_kl_conf_list = [np.mean((rkhs_kl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_kl_list[:, 4])), np.mean((rkhs_kl_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_kl_list[:, 6])), np.mean((rkhs_kl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_kl_list[:, 8]))]
rkhs_tl_conf_list = [np.mean((rkhs_tl_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_tl_list[:, 4])), np.mean((rkhs_tl_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_tl_list[:, 6])), np.mean((rkhs_tl_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_tl_list[:, 8]))]
rkhs_mle_conf_list = [np.mean((rkhs_mle_list[:, 3] <= true_ATEs_list) * (true_ATEs_list <= rkhs_mle_list[:, 4])), np.mean((rkhs_mle_list[:, 5] <= true_ATEs_list) * (5 <= rkhs_mle_list[:, 6])), np.mean((rkhs_mle_list[:, 7] <= true_ATEs_list) * (true_ATEs_list <= rkhs_mle_list[:, 8]))]


In [ ]:
results2 = np.array([
                  nn_ls_conf_list,
                  nn_kl_conf_list,
                  nn_tl_conf_list,
                  nn_mle_conf_list,
                  rkhs_ls_conf_list,
                  rkhs_kl_conf_list,
                  rkhs_tl_conf_list,
                  rkhs_mle_conf_list])

In [ ]:
results2

In [ ]:
mse_kl = (nn_kl_list[:, :3] - np.array([true_ATEs_list]*3).T)**2

In [ ]:
print(pd.DataFrame([results.flatten(), results2.flatten()]).to_latex(float_format="%.2f"))